In [1]:
import random
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
import sktime
from sklearn.linear_model import RidgeClassifierCV
from imblearn.over_sampling import SMOTE
from sktime.datatypes._panel._convert import from_2d_array_to_nested
from sktime.transformations.panel.rocket import Rocket
import lightgbm as lgbm
from sklearn.metrics import classification_report,accuracy_score,precision_score,recall_score


In [2]:
df = pd.read_csv('reconstructed_data.csv')

X = df.drop('label',axis=1)
Y = df['label']

Y

0       0
1       0
2       0
3       0
4       0
       ..
7995    9
7996    9
7997    9
7998    9
7999    9
Name: label, Length: 8000, dtype: int64

In [3]:
tscv = sk.model_selection.TimeSeriesSplit(n_splits=2)

# Use the TimeSeriesSplit object to split your data
for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]

In [4]:
Y_train

0       0
1       0
2       0
3       0
4       0
       ..
5329    6
5330    6
5331    6
5332    6
5333    6
Name: label, Length: 5334, dtype: int64

In [5]:
Y_test

5334    6
5335    6
5336    6
5337    6
5338    6
       ..
7995    9
7996    9
7997    9
7998    9
7999    9
Name: label, Length: 2666, dtype: int64

In [6]:
smote = SMOTE(sampling_strategy='minority')
X_sm,Y_sm = smote.fit_resample(X_train,Y_train)
X_sm = from_2d_array_to_nested(X_sm)

In [7]:
rocket = Rocket(num_kernels=10,n_jobs=-1)  # by default, ROCKET uses 10,000 kernels
rocket.fit(X_sm)
X_train_transform = rocket.transform(X_sm)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [9]:
classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10))
classifier.fit(X_train_transform, Y_sm)

RidgeClassifierCV(alphas=array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]))

In [10]:
X_test = from_2d_array_to_nested(X_test)
X_test_transform = rocket.transform(X_test)

In [12]:
Y_pred = classifier.predict(X_test_transform)
print(classification_report(Y_pred,Y_test))

              precision    recall  f1-score   support

           0       1.00      0.12      0.22      2111
           1       0.19      0.98      0.32        50
           2       0.00      0.00      0.00         0
           3       0.01      0.18      0.01        11
           4       0.05      0.09      0.06       133
           5       0.05      0.50      0.10        28
           6       0.20      0.61      0.30        89
           7       0.31      0.39      0.34       225
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00        19

    accuracy                           0.18      2666
   macro avg       0.18      0.29      0.14      2666
weighted avg       0.83      0.18      0.22      2666



/opt/miniconda3/envs/bigdata/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/envs/bigdata/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/miniconda3/envs/bigdata/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
